# Titanicデータセットを用いた機械学習入門

<div style="display:flex;justify-content:center;">
    <img src="./assets/RMS_Titanic_3.jpg"  width="600px" alt="https://commons.wikimedia.org/wiki/File:RMS_Titanic_3.jpg" />
</div>

今回は、データサイエンスや機械学習を学ぶものの多くが通る道である、Titanicデータセットを用いたデータ分析を行っていく。

もちろんタイタニックは、映画「タイタニック」で有名なあの船。

> タイタニック（英語: RMS Titanic、ロイヤルメールシップ・タイタニック）は、20世紀初頭に建造されたイギリス船籍のオーシャン・ライナー。
> ホワイト・スター・ライン社が保有するオリンピック級客船の2番船であったが、処女航海中の1912年4月14日深夜に氷山に衝突し、その際の損傷による浸水が原因となって翌15日未明に沈没した。([wikipediaより](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF_(%E5%AE%A2%E8%88%B9)))

## Titanic dataset について

Titanic datasetは色々なところでフリーのデータセットとして公開されている。
- Kaggle (https://www.kaggle.com/competitions/titanic)
- Tensorflow Datasets (https://www.tensorflow.org/datasets/catalog/titanic?hl=en)
- Seaborn Datasets (https://github.com/mwaskom/seaborn-data)
などなど

今回は、 Kaggleのものを利用する。

## 機械学習プロジェクトの一般的な流れ

1. 分析の目的と問題設定
2. データを取得する
3. EDA ~ データからインサイトを得る ~
4. 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~
5. モデルの作成、学習、推論の実行
<br>↑今回はここまで
6. モデルのファインチューニング
7. 結果の提示
8. システムに組み込む

# 必要なライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## データを取得する

早速データを取得してみる。

In [ ]:
test_set = pd.read_csv('https://raw.githubusercontent.com/KOHSUK/data-science-cource/master/datasets/titanic/test.csv')
train_set = pd.read_csv('https://raw.githubusercontent.com/KOHSUK/data-science-cource/master/datasets/titanic/train.csv')

In [ ]:
# データを頭から数行だけ見てみる
train_set.head() # head(10)とすると10行取得できる

In [ ]:
train_set.shape

In [ ]:
print('train:', train_set.columns)
print('test:', test_set.columns)

test_setには`Survived`が存在しない。

各列の説明はこの通り

| 列名 | 説明 |
|------|------|
| PassengerId | 乗客のID |
| Survived | 生存したか否か (0 = No, 1 = Yes) |
| Pclass | チケットのクラス (1 = 1st, 2 = 2nd, 3 = 3rd) |
| Name | 乗客の名前 |
| Sex | 性別 |
| Age | 年齢 |
| SibSp | タイタニック号に乗船していた兄弟/配偶者の数 |
| Parch | タイタニック号に乗船していた親/子供の数 |
| Ticket | チケット番号 |
| Fare | 旅客運賃 |
| Cabin | キャビン番号 |
| Embarked | 乗船港 (C = Cherbourg, Q = Queenstown, S = Southampton) |


train_setのデータを用いて、各列の情報からSurvivedを予測するモデルを作成する。

test_setのデータのそれぞれの行のSurvivedの結果を予測する。

という流れになる。

## 注意点

Kaggle の test.csvの真の答えは、Kaggleが持っていて、Kaggle上で予測結果を提出することでしか、最終的な予測の精度は測れない（Web上に答えは転がっているだろうが)。

ここでは、train.csvの一部のデータ(例えば全体の20%)を答えがわからないふりをして傍に置いておき、

もう一方のデータを使って学習、とっておいたデータ(から真の`Survived`を排除したデータ)で予測を行い、精度を測ることにする。

以下はそのために、データを分けている。

In [ ]:
train_data, test_data = train_test_split(train_set, test_size=0.2)

# Survived列のみを取り出したデータを作成する
test_survived = test_data[["Survived"]]
# Survived列のみを削除したデータを作成する
test_data = test_data.drop('Survived', axis=1)

# 処理結果を表示して、意図通りの列ができているか確認する。
print('train_data:', train_data.columns)
print()
print('test_data:', test_data.columns)
print()
print('test_survived:', test_survived.columns)

これで、

train_data ・・・ 学習用のデータ<br>
test_data ・・・ 検証用のデータ<br>
test_survived ・・・ 検証用のデータの答え(Survived)

に分けることができた。

## EDA ~ データからインサイトを得る ~

ここから実際に、分析を行っていく。

まずは、学習用データの中身がどのようなものなのか改めて確認していく。

基本として注目する点は、

1. データ自体のを眺める
2. 欠損値の確認
3. データの特徴を把握する
4. 異常値(外れ値)の有無
5. 数値列間の相関係数
6. その他ドメイン知識や仮説に基づいたデータの探索

ちなみに、データの特徴を掴み、モデル作成のインサイトを得るためにデータをさまざまな角度から見ていくことを、<br>
EDA（Explanatory Data Analysis, 探索データ分析）という。

### 1. データ自体を眺める

#### データセットの中身を少し眺める

In [ ]:
train_data.head()

In [ ]:
test_data.head()

ちなみにtrain_data自体のデータ型を調べると、、、

In [ ]:
type(train_data)

Pandasというライブラリの`DataFrame`という型となっている。

<公式のドキュメント><br>
https://pandas.pydata.org/pandas-docs/stable/reference/frame.html

#### データセットのサイズを確認する

DataFrame.shape　で　`(行数, 列数)`という出力を得られる。

In [ ]:
train_data.shape

In [ ]:
test_data.shape

### 2. 欠損値の確認

データの中には何らかの理由で、特定の列の値が取得できない場合がある。

その場合、データセットの中に`欠損値`が存在することになる。

例えば、`Age`(年齢)列に欠損値がある場合、そのままでは全体の平均年齢を求めることはできない。

よって、欠損値がデータセットに含まれるのかどうかは大きな問題となる。

In [ ]:
# 各列の欠損値の数の合計を求める
train_data.isna().sum()

`Age`, `Cabin`, `Embarked`については、欠損値が含まれるので何らかの対処をしなければならないことがわかった。

### 3. データの特徴を把握する

#### 各列のデータ型を確認する

`DataFrame.dtypes`で各列のデータ型を確認できる。

In [ ]:
train_data.dtypes

`int64`, `float64`は数値データ
`object`はPythonのobject型。String(文字列)はobjectの一種。

#### 数値データの基本統計量を確認する

`DataFrame.describe()`で各数値列の基本統計量を自動的に計算してくれる。

In [ ]:
train_data.describe()

数値以外についても`describe`を呼ぶと、最頻値などを確認できる。

例えば、`Fare`を確認すると、中央値(50%)が14前後に対して平均が30くらい、<br>
最大値は75%点の31くらいから大きく開いて500をこえる値となっている。

`Fare`に外れ値が存在することを疑うことができる。

In [ ]:
train_data.describe(exclude='number')

#### カテゴリ値確認する

データの説明などからどれがカテゴリデータかを確認する。

In [ ]:
# 全て文字列に変換してから、`describe()`を呼ぶと数値データについてもカテゴリ値かどうか確認できる
train_data.astype('str').describe()

`unique`行の数値が少ないものはカテゴリ値の場合が多い。

今回は、`Survived`, `Pclass`, `Sex`, `Parch`, `Embarked`は明らかにカテゴリ値。

### 4. 異常値(外れ値)を確認する

異常値(外れ値)は、分析結果に大きな影響を与える。

データを可視化することで異常値に気づきやすくなる。

以下では例として、`describe()`で確認した時に外れ値を疑った`Fare`のヒストグラムを描画してみる。

In [ ]:
sns.displot(train_data['Fare'], kde=True, rug=False, bins=10)
plt.title('Fare')
plt.tight_layout()
plt.show()


In [ ]:
train_data[train_data['Fare'] > 400].head()

異常値、外れ値が何らかの想定外が起こった結果（入力ミスとか計測器の故障とか）であれば、

- その行自体を学習データから省く
- 何か別のデータに置き換える

などの処理を行う必要がある。

一方で、外れ値になっていること自体に意味がある場合もある。

例えばここでは、先に「運賃を多く払っているものは優先的に救命ボートに乗っているのではないか」という仮説を検証する必要がある。

### 5. その他ドメイン知識や仮説に基づいたデータの探索

#### 仮説「運賃を多く払っているものは優先的に救命ボートに乗っているのではないか」を検証する

In [ ]:
# データをコピーしておく
check = train_data.copy()

# 運賃を10ごとのカテゴリ値に変換した列を作成する(`// 10`は10で割った時の整数値でそれに10を掛けると10ごとのカテゴリになる)
# 例：　167 // 10 * 10 = 16 * 10 = 160
check['FareCat'] = check['Fare'] // 10 * 10
# 列数カウント用ダミー値
check['Count'] = 1

# 運賃カテゴリごとの生存率を求める
fare_survived = check.groupby('FareCat').agg({'Survived': 'mean', 'Count': 'sum'})

fare_survived

Fareが70未満のものは生存率が低いことがわかる。

人数に差があるので一口では言えないが、運賃によって生存率が変わると言えるかもしれない。

高い運賃を払える人は社会的階級が高く、救命ボートに優先的に乗せられた（のかもしれない）。

#### チケットの等級ごとに生存率を確認する

In [ ]:
pclass_survived = check.groupby('Pclass').agg({'Survived': 'mean', 'Count': 'sum'})
pclass_survived

明らかにチケットの等級の高いものの生存率が高い。

#### 性別、年齢で生存率が変わるのか確認する

Wikipediaのタイタニックのページには、「[ライトラーは一等船客の「女性と子供を優先する」ことを遵守した](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF_(%E5%AE%A2%E8%88%B9)#:~:text=%E3%83%A9%E3%82%A4%E3%83%88%E3%83%A9%E3%83%BC%E3%81%AF%E4%B8%80%E7%AD%89%E8%88%B9%E5%AE%A2%E3%81%AE%E3%80%8C%E5%A5%B3%E6%80%A7%E3%81%A8%E5%AD%90%E4%BE%9B%E3%82%92%E5%84%AA%E5%85%88%E3%81%99%E3%82%8B%E3%80%8D%E3%81%93%E3%81%A8%E3%82%92%E9%81%B5%E5%AE%88%E3%81%97%E3%81%9F)」との記載がある。

データ上その事実が読み取れるのか確認してみる。

In [ ]:
# 性別ごとの生存率
sex_survived = check.groupby('Sex').agg({'Survived': 'mean', 'Count': 'sum'})
sex_survived

圧倒的に女性の生存率が高い。

In [ ]:
# 年齢カテゴリを作成
check['AgeCat'] = check['Age'] // 10 * 10

age_survived = check.groupby('AgeCat').agg({'Survived': 'mean', 'Count': 'sum'})
age_survived

10歳未満の子供は生存率が高いことがわかる。

In [ ]:
# 両方まとめて集計してみる
age_sex_survived = check.groupby(['AgeCat', 'Sex']).agg({'Survived': 'mean', 'Count': 'sum'})
age_sex_survived

#### 仮説「敬称によって生存率が異なる」

時代的にも敬称は地位を表すことがあるため、`Name`列の敬称に何が使われているかを利用できるかもしれない。


In [ ]:
def find_matching_substring(big_string, substrings):
    # check if any substring is present in the big string
    big_string = str(big_string)
    matches = [substring for substring in substrings if substring in big_string]

    # return the first matching substring, or np.nan if none found
    if matches:
        return matches[0]
    else:
        return np.nan

In [ ]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

check['Title'] = train_data['Name'].map(lambda x: find_matching_substring(x, title_list))
check.head()

In [ ]:
# 欠損値のチェック
check['Title'].isnull().sum()

In [ ]:
# 敬称ごとの生存率を集計する
title_survived = check.groupby('Title').agg({'Survived': 'mean', "Count": 'sum'})
title_survived

男性につける敬称のうち、`Dr`, `Master`, `Mr`を比べると、

`Master` > `Dr` > `Mr` の順に生存率が高いことがわかる。

## Tips: 率を平均(mean)で求める理由

先ほどから、生存率を求める計算方法に'mean'(平均)を指定している。

これは生存したか否かの列`Survived`が0, 1の2値であることを利用している。

生存率を求める方法は、

$$
    生存率 = \frac{Survived=1の数}{全体の人数}
$$

となる。

一方で、`Survived`が0, 1の2値であるときのSurvivedの平均は

$$
    Survivedの平均 = \frac{全行のSurvived(0,1)の合計}{行数}
$$

であり、データの一行一行が一人一人の人なので、`行数`は`全体の人数`と同値だし、<br>
0 or 1 をとるデータの合計は1の数を数えているのと同じである。

よって、

$$
    Survivedの平均 = \frac{全行のSurvived(0,1)の合計}{行数} = \frac{Survived=1の数}{全体の人数} = 生存率
$$

と言える。

## 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~

今度は実際に、データを使って機械学習モデルを学習させるための前準備として、データの処理を行っていきます。

#### テストデータと訓練データを結合する。

まず、テストデータ(test_data)と訓練データ(train_data)を結合します。
また、後で分割が容易になるように元々テストデータなのかどうかのフラグ列も追加します(`isTest` 0: テストデータではない、1: テストデータである)

せっかく最初の時点で、分割したものをなぜまた結合するのかと思うだろうが、<br>
こうすることで、例えば欠損値を埋める処理を`train_data`と`test_data`の2回行う必要がなくなるなど、<br>
前処理を都合よく行うことができる。

In [ ]:
col_name = 'isTest'

# 以下のようにすることで、全行に1(または0)を入れることができる。
test_data[col_name] = 1
train_data[col_name] = 0

In [ ]:
test_data.head()

In [ ]:
train_data.head()

isTestに意図通りの値が入っている。

In [ ]:
# train_dataとtest_dataを結合する

dat = pd.concat([train_data, test_data])

In [ ]:
# 行数を確認して結合ができているか確認する(shapeは(行数, 列数)なので、shape[0]は行数を取得できる)
print("train_data + test_data", train_data.shape[0] + test_data.shape[0])
print("dat", dat.shape[0])


#### 欠損値の処理

`Age`, `Cabin`, `Embarked`に欠損値があるため、何らかの処理をする必要があります。

欠損値の処理方法は、利用するモデルやデータの性質などによる。

ここでは、`Age`は中央値、`Embarked`は最頻値で欠損値を埋めることにする。

簡単にするため、`Cabin`は推論に用いないことにし、欠損値はそのままにしておくことにする。

In [ ]:
mean_age = dat['Age'].mean()
top_embarked = dat[['Embarked']].describe(exclude='number').at['top', 'Embarked']

print(mean_age)
print(top_embarked)

In [ ]:
# それぞれ欠損値を埋める(fillnaを用いる)
dat['Age'].fillna(mean_age, inplace=True)
dat['Embarked'].fillna(top_embarked, inplace=True)

# 確認する
dat[['Age', 'Embarked']].isna().sum()

#### 異常値の処理

異常値もここで処理をすることになる。

`Fare`に外れ値が含まれることがわかっているが、等級が生存率に影響を与えていることがわかっているため、

今回は特に処理を行わないことにする。


#### 敬称を特徴量に加える

先ほど、敬称ごとに生存率に差が出る可能性があるとわかったため、それを特徴量に含める

In [ ]:
dat['Title'] = dat['Name'].map(lambda x: find_matching_substring(x, title_list))
dat[['Title']].isnull().sum()

#### カテゴリ値の変換

多くの機械学習モデルで、文字列データより数値データが好ましい場合がある。（文字列で問題ないモデルもあるが）

また、カテゴリ値をダミー変数とする方が良い場合、しなくても良い場合などさまざまではある。

そのあたりは、モデルやデータの性質によるのだが、

ここでは`pandas.get_dummies`を用いてダミー変数に変換してみる。

ダミー変数とは？<br>
https://bellcurve.jp/statistics/glossary/1538.html

In [ ]:
dat = pd.get_dummies(dat, drop_first=True, columns=['Sex', 'Embarked', 'Title'])

In [ ]:
# 最終的な列

dat.columns

In [ ]:
dat.head()

## モデルの作成、学習、推論の実行

欠損値や外れ値の処理をし、新しい特徴量を作成したので、
早速モデルを作成し、学習・推論して結果を見てみる。

In [ ]:
# 前処理をしたデータを、 学習データ、学習データの教師データ、検証データに分ける。

train = dat[dat['isTest'] == 0].copy()
test = dat[dat['isTest'] ==1 ].copy()

columns = ['Pclass', 'Age', 'SibSp', 'Parch',
       'Fare', 'Sex_male', 'Embarked_Q',
       'Embarked_S', 'Title_Col', 'Title_Countess', 'Title_Don', 'Title_Dr',
       'Title_Jonkheer', 'Title_Major', 'Title_Master', 'Title_Miss',
       'Title_Mlle', 'Title_Mme', 'Title_Mr', 'Title_Mrs', 'Title_Ms',
       'Title_Rev']

X_train = train[columns]
y_train = train['Survived']
X_test = test[columns]

In [ ]:
# Random Forests を利用する

from sklearn.ensemble import RandomForestClassifier

# ハイパーパラメータは一旦適当な値にしている
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1, random_state=79)

# 学習を行う
rnd_clf.fit(X_train, y_train)

これで学習が完了！

このモデルを使って推論をしてみる。

In [ ]:
y_pred_rf = rnd_clf.predict(X_test)

正解率を求めると、

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_survived['Survived'], y_pred_rf)